# Data

We obtained our datasets from the USDA Economic Research Service and from The Educational Opportunity Project at Stanford. These datasets will subsequently be referred to as the Food dataset and the Education dataset respectively. In this notebook, we prepare our analysis dataset by reading in the data and merging the two datasets.

In addition to these 2 datasets, we used a small dataset from the below github URL to convert state names to state abbreviations. We needed to perform this conversion to make merging on state and county possible.

In [1]:
#@title [Hidden Code] State Abbreviation codes from https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [2]:
import pandas as pd
import numpy as np

#from google.colab import drive 
#drive.mount('/content/gdrive')
education_df = pd.read_csv("../data/seda_county_pool_gcs_v30.csv")

## Education Dataset

First, we import the Education dataset. The Educational Opportunity Project at Stanford has a variety of datasets to choose from. Because our Food dataset is county-level, we chose an Education dataset that also has county-level observations. We chose the GCS pooled version. The pooled version contains "pooled overall" observations which means they contain estimates that are averaged across grades, years, and subjects. Since we want to analyze county level educational performance we did not see a need to use data at a per subject per grade level of detail. GCS stands for grade cohort standardized estimates.

In [3]:
education_df.head()

,countyid,countyname,fips,stateabb,subgroup,gap_est,tot_asmts,cellcount,mn_asmts,mn_avg_ol,...,mn_coh_eb,mn_mth_eb,mn_avg_eb_se,mn_grd_eb_se,mn_coh_eb_se,mn_mth_eb_se,mn_avg_ol_se,mn_grd_ol_se,mn_coh_ol_se,mn_mth_ol_se
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,...,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091
1,1001,Autauga County,1,AL,asn,0,987,82,12.036585,6.959411,...,NaN,NaN,0.132585,NaN,NaN,NaN,0.134268,NaN,NaN,NaN
2,1001,Autauga County,1,AL,blk,0,17213,96,179.302083,4.000924,...,-0.045586,-0.333673,0.040307,0.028377,0.017289,0.078631,0.040381,0.029171,0.017608,0.080899
3,1001,Autauga County,1,AL,ecd,0,29288,84,348.666667,4.455159,...,-0.000746,-0.322555,0.038907,0.028050,0.018347,0.074120,0.039059,0.028976,0.018917,0.076113
4,1001,Autauga County,1,AL,fem,0,31568,84,375.809524,5.819313,...,-0.085387,-0.634774,0.039181,0.026747,0.016382,0.074575,0.039311,0.027666,0.016879,0.076284


From the Education dataset, we drop all of the observations that contain information split out by demographic subgroups. By selecting the all category, we retain only the estimates for all demographic groups combined. This enables our dataset to have a single observation per county which makes merging and analyzing the data more streamlined.

> Indented block



In [4]:
# Only grab overall data per county
education_df = education_df[education_df.subgroup == "all"]
education_df.head()

,countyid,countyname,fips,stateabb,subgroup,gap_est,tot_asmts,cellcount,mn_asmts,mn_avg_ol,...,mn_coh_eb,mn_mth_eb,mn_avg_eb_se,mn_grd_eb_se,mn_coh_eb_se,mn_mth_eb_se,mn_avg_ol_se,mn_grd_ol_se,mn_coh_ol_se,mn_mth_ol_se
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,...,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091
15,1003,Baldwin County,1,AL,all,0,213214,96,2220.979167,5.428993,...,-0.048984,-0.593051,0.030970,0.022101,0.013229,0.060861,0.030998,0.022601,0.013487,0.061982
30,1005,Barbour County,1,AL,all,0,26521,96,276.260417,3.692509,...,0.123856,-0.717099,0.035750,0.025260,0.015315,0.069926,0.035792,0.026005,0.015718,0.071609
44,1007,Bibb County,1,AL,all,0,25678,96,267.479167,4.235791,...,0.020140,-0.846959,0.036446,0.025803,0.015509,0.071291,0.036493,0.026596,0.015929,0.073065
56,1009,Blount County,1,AL,all,0,71940,96,749.375000,4.689641,...,0.088930,-0.980552,0.032432,0.023091,0.013791,0.063648,0.032465,0.023661,0.014084,0.064929


## Food Dataset

Next, we import the Food dataset. This dataset contains food desert measurements at a census tract level of observation. 

In [5]:
food_df = pd.read_csv("../data/Food_deserts_in_the_US.csv")
food_df.head()

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,...,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,...,214,888,1217,5,0,5,55,75,87,202
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,...,439,2576,647,17,5,11,117,87,108,120
3,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,...,904,4086,193,18,4,11,74,85,19,82
4,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,...,1126,8666,1437,296,9,48,310,355,198,488


# Data Aggregation

The Food Desert dataset splits counties into 'tracts' which split a county into smaller units. These tracts are what the food desert data was collected on. Thus, we need to aggregate tracts together in order to have data for an entire county. Each column for a tract can be classified as a total, percentage, or boolean value, with a few exceptions.

In [6]:
other_colns  = ['CensusTract',
                'State',
                'County',
                'MedianFamilyIncome'] 

bool_columns = ['Urban',
                'GroupQuartersFlag',
                'LILATracts_1And10',
                'LILATracts_halfAnd10',
                'LILATracts_1And20',
                'LILATracts_Vehicle',
                'HUNVFlag',
                'LowIncomeTracts',
                'LA1and10',
                'LAhalfand10',
                'LA1and20',
                'LATracts_half',
                'LATracts1',
                'LATracts10',
                'LATracts20',
                'LATractsVehicle_20']

tot_columns  = ['POP2010',
                'OHU2010',
                'NUMGQTRS',
                'LAPOP1_10',
                'LAPOP05_10',
                'LAPOP1_20',
                'LALOWI1_10',
                'LALOWI05_10',
                'LALOWI1_20',
                'lapophalf',
                'lalowihalf',
                'lakidshalf',
                'laseniorshalf',
                'lawhitehalf',
                'lablackhalf',
                'laasianhalf',
                'lanhopihalf',
                'laaianhalf',
                'laomultirhalf',
                'lahisphalf',
                'lahunvhalf',
                'lasnaphalf',
                'lapop1',
                'lalowi1',
                'lakids1',
                'laseniors1',
                'lawhite1',
                'lablack1',
                'laasian1',
                'lanhopi1',
                'laaian1',
                'laomultir1',
                'lahisp1',
                'lahunv1',
                'lasnap1',
                'lapop10',
                'lalowi10',
                'lakids10',
                'laseniors10',
                'lawhite10',
                'lablack10',
                'laasian10',
                'lanhopi10',
                'laaian10',
                'laomultir10',
                'lahisp10',
                'lahunv10',
                'lasnap10',
                'lapop20',
                'lalowi20',
                'lakids20',
                'laseniors20',
                'lawhite20',
                'lablack20',
                'laasian20',
                'lanhopi20',
                'laaian20',
                'laomultir20',
                'lahisp20',
                'lahunv20',
                'lasnap20',
                'TractLOWI',
                'TractKids',
                'TractSeniors',
                'TractWhite',
                'TractBlack',
                'TractAsian',
                'TractNHOPI',
                'TractAIAN',
                'TractOMultir',
                'TractHispanic',
                'TractHUNV',
                'TractSNAP',]

pct_columns  = ['PCTGQTRS',
                'PovertyRate',
                'lapophalfshare',
                'lalowihalfshare',
                'lakidshalfshare',
                'laseniorshalfshare',
                'lawhitehalfshare',
                'lablackhalfshare',
                'laasianhalfshare',
                'lanhopihalfshare',
                'laaianhalfshare',
                'laomultirhalfshare',
                'lahisphalfshare',
                'lahunvhalfshare',
                'lasnaphalfshare',
                'lapop1share',
                'lalowi1share',
                'lakids1share',
                'laseniors1share',
                'lawhite1share',
                'lablack1share',
                'laasian1share',
                'lanhopi1share',
                'laaian1share',
                'laomultir1share',
                'lahisp1share',
                'lahunv1share',
                'lasnap1share',
                'lapop10share',
                'lalowi10share',
                'lakids10share',
                'laseniors10share',
                'lawhite10share',
                'lablack10share',
                'laasian10share',
                'lanhopi10share',
                'laaian10share',
                'laomultir10share',
                'lahisp10share',
                'lahunv10share',
                'lasnap10share',
                'lapop20share',
                'lalowi20share',
                'lakids20share',
                'laseniors20share',
                'lawhite20share',
                'lablack20share',
                'laasian20share',
                'lanhopi20share',
                'laaian20share',
                'laomultir20share',
                'lahisp20share',
                'lahunv20share',
                'lasnap20share'] 

We split the dataframe into three frames each containing a different kind of data. 

In [7]:
pct_df = food_df[['State', 'County', 'POP2010'] + pct_columns]
bool_df = food_df[['State', 'County', 'POP2010'] + bool_columns]
total_df = food_df[['State', 'County'] + tot_columns]

We have some helper functions for aggregating different kinds of data:

`find_pop_pct` takes a dataframe of tracts and finds the percentage of the total population that each tract contains. 

`agg_pct` takes a dataframe of tracts and first uses `find_pop_pct` to get the ratio of the population each tract has. Then it multiplies each data point for each tract by the tract's population percentage and then adds those together to get the percentage for the entire county.

`agg_bool` works exactly the same as `agg_pct`, but since we're working with booleans, the final result reads more like 'the percentage of population for which the property is true' rather than finding a new boolean for the entire county. 

`agg_income` returns the average of the median income for each tract in a county. 

In [8]:
def find_pop_pct(df):
    tot_pop = df['POP2010'].sum()
    pop_pct = df['POP2010'] / tot_pop
    
    return pop_pct

def agg_pct(df):
    #print (df.columns)
    pp_df = find_pop_pct(df)
    return df[pct_columns].mul(pp_df, 0).sum()
    
def agg_bool(df):
    pp_df = find_pop_pct(df)
    return df[bool_columns].mul(pp_df, 0).sum()

def agg_income(df):
    pp_df = find_pop_pct(df)
    return df[['MedianFamilyIncome']].mul(pp_df, 0).sum()

We use the helper functions to appropriately aggregate the data for each county. 

In [9]:
total_agg = total_df.groupby(['State', 'County']).sum().reset_index()
pct_agg = pct_df.groupby(['State', 'County']).apply(lambda x: agg_pct(x)).reset_index()
bool_agg = bool_df.groupby(['State', 'County']).apply(lambda x: agg_bool(x)).reset_index()
income_agg = food_df[['State', 'County', 'POP2010', 'MedianFamilyIncome']].groupby(['State', 'County']).apply(lambda x: agg_income(x)).reset_index()

Finally, we reassemble the four frames. 

In [10]:
food_df = total_agg.join(income_agg.drop(['State', 'County'], axis=1)).join(pct_agg.drop(['State', 'County'], axis=1)).join(bool_agg.drop(['State', 'County'], axis=1))

Since the Food dataset contains full state names and the Education dataset contains state abbreviations, we add a variable to the Food dataset that contains the state abbreviations. This will enable us to later merge the datasets by state. 

In [11]:
food_df['stateabb'] = food_df.apply(lambda row: us_state_abbrev[row['State']], axis=1)

food_df['stateabb']

0       AL
1       AL
2       AL
3       AL
4       AL
        ..
3136    WY
3137    WY
3138    WY
3139    WY
3140    WY
Name: stateabb, Length: 3141, dtype: object

The county names in the Education dataset have the words "County" and "Parish" as part of the county name string. add a variable to the Food dataset that removes these words from the county name. This will enable us to later merge the datasets by county.

In [12]:
def remove_county(row):
  return row['countyname'].replace(" County", "").replace(" Parish", "")

education_df['County'] = education_df.apply(remove_county, axis=1)
education_df['County']

0           Autauga
15          Baldwin
30          Barbour
44             Bibb
56           Blount
            ...    
39014    Sweetwater
39029         Teton
39041         Uinta
39055      Washakie
39065        Weston
Name: County, Length: 3134, dtype: object

## Merging

Before merging the datasets, we want to add identifiers to each variable name. We add the prefix 'Educ_' to all Education variable names and we add the prefix 'Food_' to all the Food variable names. This is done for all variables except for County and State because we will need to merge on these variable and in order to do so, they need to have the same name.

In [13]:
education_df.columns = ["Educ_" + x for x in education_df.columns]

education_df['County'] = education_df['Educ_County']
education_df['stateabb'] = education_df['Educ_stateabb']

education_df.head()

,Educ_countyid,Educ_countyname,Educ_fips,Educ_stateabb,Educ_subgroup,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,...,Educ_mn_grd_eb_se,Educ_mn_coh_eb_se,Educ_mn_mth_eb_se,Educ_mn_avg_ol_se,Educ_mn_grd_ol_se,Educ_mn_coh_ol_se,Educ_mn_mth_ol_se,Educ_County,County,stateabb
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,...,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL
15,1003,Baldwin County,1,AL,all,0,213214,96,2220.979167,5.428993,...,0.022101,0.013229,0.060861,0.030998,0.022601,0.013487,0.061982,Baldwin,Baldwin,AL
30,1005,Barbour County,1,AL,all,0,26521,96,276.260417,3.692509,...,0.025260,0.015315,0.069926,0.035792,0.026005,0.015718,0.071609,Barbour,Barbour,AL
44,1007,Bibb County,1,AL,all,0,25678,96,267.479167,4.235791,...,0.025803,0.015509,0.071291,0.036493,0.026596,0.015929,0.073065,Bibb,Bibb,AL
56,1009,Blount County,1,AL,all,0,71940,96,749.375000,4.689641,...,0.023091,0.013791,0.063648,0.032465,0.023661,0.014084,0.064929,Blount,Blount,AL


In [14]:
food_df.columns = ["Food_" + x for x in food_df.columns]

food_df['County'] = food_df['Food_County']
food_df['stateabb'] = food_df['Food_stateabb']

food_df.head()

,Food_State,Food_County,Food_POP2010,Food_OHU2010,Food_NUMGQTRS,Food_LAPOP1_10,Food_LAPOP05_10,Food_LAPOP1_20,Food_LALOWI1_10,Food_LALOWI05_10,...,Food_LAhalfand10,Food_LA1and20,Food_LATracts_half,Food_LATracts1,Food_LATracts10,Food_LATracts20,Food_LATractsVehicle_20,Food_stateabb,County,stateabb
0,Alabama,Autauga,54571,20221,455,18092.661348,30141.638549,11064.872922,6835.642497,10632.847467,...,0.752323,0.494695,0.534460,0.494695,0.217863,0.0,0.367558,AL,Autauga,AL
1,Alabama,Baldwin,182265,73180,2307,46400.350641,72307.849152,43486.806226,15459.848416,23336.898273,...,0.477223,0.430768,0.453976,0.430768,0.023246,0.0,0.131622,AL,Baldwin,AL
2,Alabama,Barbour,27457,9820,3193,6683.975691,9302.717960,1182.176872,3251.362574,4750.994136,...,0.579087,0.241760,0.241760,0.241760,0.337327,0.0,0.478931,AL,Barbour,AL
3,Alabama,Bibb,22915,7953,2224,295.614426,295.614426,0.000000,164.903295,164.903295,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.382588,AL,Bibb,AL
4,Alabama,Blount,57322,21578,489,5855.939161,7387.084978,3900.420369,2570.900481,3231.158784,...,0.275304,0.122989,0.122989,0.122989,0.152315,0.0,0.387931,AL,Blount,AL


We can now merge the data by County and State. Because the Food dataset has multiple observations per year, it can be seen in the table below that there is repeated education information per observation. We will handle this in a few steps.

In [28]:
combined_df = pd.merge(education_df, food_df, on=["County", "stateabb"])

combined_df.head()

,Educ_countyid,Educ_countyname,Educ_fips,Educ_stateabb,Educ_subgroup,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,...,Food_LowIncomeTracts,Food_LA1and10,Food_LAhalfand10,Food_LA1and20,Food_LATracts_half,Food_LATracts1,Food_LATracts10,Food_LATracts20,Food_LATractsVehicle_20,Food_stateabb
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,...,0.113815,0.712558,0.752323,0.494695,0.534460,0.494695,0.217863,0.0,0.367558,AL
1,1003,Baldwin County,1,AL,all,0,213214,96,2220.979167,5.428993,...,0.193888,0.454015,0.477223,0.430768,0.453976,0.430768,0.023246,0.0,0.131622,AL
2,1005,Barbour County,1,AL,all,0,26521,96,276.260417,3.692509,...,0.768438,0.579087,0.579087,0.241760,0.241760,0.241760,0.337327,0.0,0.478931,AL
3,1007,Bibb County,1,AL,all,0,25678,96,267.479167,4.235791,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.382588,AL
4,1009,Blount County,1,AL,all,0,71940,96,749.375000,4.689641,...,0.434510,0.275304,0.275304,0.122989,0.122989,0.122989,0.152315,0.0,0.387931,AL


Since there is repeated county and state information in our newly merged dataset, we will now drop the excess columns. We also drop the subgroup column as its the same for all counties and provides no information. 

In [29]:
combined_df.drop(["Food_State", "Food_County", "Food_stateabb", "Educ_stateabb", "Educ_County", "Educ_countyname", "Educ_subgroup"], axis=1, inplace=True)
combined_df

,Educ_countyid,Educ_fips,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,Educ_mn_grd_ol,Educ_mn_coh_ol,Educ_mn_mth_ol,...,Food_HUNVFlag,Food_LowIncomeTracts,Food_LA1and10,Food_LAhalfand10,Food_LA1and20,Food_LATracts_half,Food_LATracts1,Food_LATracts10,Food_LATracts20,Food_LATractsVehicle_20
0,1001,1,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,...,0.367558,0.113815,0.712558,0.752323,0.494695,0.534460,0.494695,0.217863,0.000000,0.367558
1,1003,1,0,213214,96,2220.979167,5.428993,0.832481,-0.051916,-0.611446,...,0.131622,0.193888,0.454015,0.477223,0.430768,0.453976,0.430768,0.023246,0.000000,0.131622
2,1005,1,0,26521,96,276.260417,3.692509,0.956793,0.129474,-0.743482,...,0.478931,0.768438,0.579087,0.579087,0.241760,0.241760,0.241760,0.337327,0.000000,0.478931
3,1007,1,0,25678,96,267.479167,4.235791,0.996612,0.021188,-0.888118,...,0.382588,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.382588
4,1009,1,0,71940,96,749.375000,4.689641,0.866955,0.091370,-1.011810,...,0.387931,0.434510,0.275304,0.275304,0.122989,0.122989,0.122989,0.152315,0.000000,0.387931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,56037,56,0,33703,54,624.129630,5.529081,1.075794,0.132322,-0.057635,...,0.196297,0.196297,0.339748,0.468383,0.339748,0.420627,0.291992,0.047756,0.047756,0.244053
3053,56039,56,0,10605,54,196.388889,6.700046,1.146559,0.021997,-0.172702,...,0.000000,0.000000,0.322673,0.322673,0.322673,0.106650,0.106650,0.216023,0.216023,0.216023
3054,56041,56,0,18001,54,333.351852,6.086742,1.059938,0.037275,0.132833,...,0.308031,0.324463,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.308031
3055,56043,56,0,6159,54,114.055556,6.508509,1.023191,-0.007281,0.398879,...,0.000000,0.312317,0.389781,1.000000,0.389781,0.610219,0.000000,0.389781,0.389781,0.389781


In [33]:
combined_df.rename({'stateabb':'State'}, axis=1, inplace=True)

Our resulting dataset can be seen below. There are 3057 observations in our dataset, meaning it covers 3,057 counties. In the US there are a total of 3,141 counties. Therefore our dataset contains information for 97% of the counties in the US.

In [34]:
combined_df

,Educ_countyid,Educ_fips,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,Educ_mn_grd_ol,Educ_mn_coh_ol,Educ_mn_mth_ol,...,Food_HUNVFlag,Food_LowIncomeTracts,Food_LA1and10,Food_LAhalfand10,Food_LA1and20,Food_LATracts_half,Food_LATracts1,Food_LATracts10,Food_LATracts20,Food_LATractsVehicle_20
0,1001,1,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,...,0.367558,0.113815,0.712558,0.752323,0.494695,0.534460,0.494695,0.217863,0.000000,0.367558
1,1003,1,0,213214,96,2220.979167,5.428993,0.832481,-0.051916,-0.611446,...,0.131622,0.193888,0.454015,0.477223,0.430768,0.453976,0.430768,0.023246,0.000000,0.131622
2,1005,1,0,26521,96,276.260417,3.692509,0.956793,0.129474,-0.743482,...,0.478931,0.768438,0.579087,0.579087,0.241760,0.241760,0.241760,0.337327,0.000000,0.478931
3,1007,1,0,25678,96,267.479167,4.235791,0.996612,0.021188,-0.888118,...,0.382588,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.382588
4,1009,1,0,71940,96,749.375000,4.689641,0.866955,0.091370,-1.011810,...,0.387931,0.434510,0.275304,0.275304,0.122989,0.122989,0.122989,0.152315,0.000000,0.387931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,56037,56,0,33703,54,624.129630,5.529081,1.075794,0.132322,-0.057635,...,0.196297,0.196297,0.339748,0.468383,0.339748,0.420627,0.291992,0.047756,0.047756,0.244053
3053,56039,56,0,10605,54,196.388889,6.700046,1.146559,0.021997,-0.172702,...,0.000000,0.000000,0.322673,0.322673,0.322673,0.106650,0.106650,0.216023,0.216023,0.216023
3054,56041,56,0,18001,54,333.351852,6.086742,1.059938,0.037275,0.132833,...,0.308031,0.324463,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.308031
3055,56043,56,0,6159,54,114.055556,6.508509,1.023191,-0.007281,0.398879,...,0.000000,0.312317,0.389781,1.000000,0.389781,0.610219,0.000000,0.389781,0.389781,0.389781


In [35]:
combined_df.to_csv("../data/combined_data.csv")